# Data download

In [ ]:
!pip install curl unzip
!curl -d -O https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/projects/00ba97/Agentic+Chatbot+Assurance+Habitation+-+Processus.md data/
!curl -d -O https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/projects/00ba97/Agentic+Chatbot+Assurance+Habitation+-+Garanties.md data/
!curl -d -O https://blent-learning-user-ressources.s3.eu-west-3.amazonaws.com/projects/00ba97/attachments.zip data/
!unzip data/attachments.zip data/
!rm data/attachments.zip

In [ ]:

import re
import sys
from pathlib import Path
import json
import time


from typing import Tuple, Dict, Any, List
from typing_extensions import TypedDict

from langgraph.types import interrupt
from langgraph.graph import StateGraph, START, END

sys.path.insert(0, str(Path.cwd().parent / "src"))
from assurhabitat_agents.llm.model_loading import llm_inference
from assurhabitat_agents.config.config import TOOLS


# Declaration agent

## Tools
2. parse_declaration
3. verify_completeness
4. ask_human


In [ ]:
class DeclarationReActState(TypedDict):
    question: str  # La question initiale de l'utilisateur
    history: list[str]  # L'historique des échanges (Thought, Action, Observation)
    last_action: str | None  # Le nom de l'outil à appeler (si applicable)
    last_arguments: dict | None  # Les arguments à passer à l'outil
    last_observation: str | None  # Le résultat de l'outil appelé
    is_complete: bool | None  # La réponse finale
    parsed_declaration: dict | None # Stockage json de la declaration parser par le tool
    missing: list[str] | None # champs manquant dans la declarations

## Nodes
1. thought
2. execute

In [ ]:
def format_prompt(state: DeclarationReActState, tools) -> str:
    """
    Build a concise prompt for the ReAct LLM using the whole state.
    - state: the DeclarationReActState dict (contains history, parsed_declaration, missing, question, etc.)
    - actions: list of available tool names with short descriptions (e.g. ["parse_declaration", "verify_completeness", "ask_human"])
    The function returns a prompt string ready to be sent to the LLM.
    """
    # Keep prompt short: only include last few history entries
    HISTORY_KEEP = 10
    history = state.get("history", [])[-HISTORY_KEEP:]

    # Show parsed_declaration and missing fields if available
    parsed = state.get("parsed_declaration")
    missing = state.get("missing", [])

    # Build actions block
    actions_block = "\n".join(f"- {a}" for a in tools) if tools else "- (no tools available)"

    parts = [
        "You are the Declaration Agent for AssurHabitat. Decide the next step: either",
        "1) call a tool (Action) OR 2) give the final answer (Réponse).",
        "",
        "Available tools:",
        actions_block,
        "",
        "Rules:",
        "- If you call a tool, use a single line: Action: TOOL_NAME",
        "- If arguments are needed, write: Arguments: then either a JSON object or key=value lines",
        "- If you return the final reply to the user, write: Réponse: <text>",
        "",
        "Context summary:",
    ]

    if state.get("question"):
        parts.append(f"Original question: {state['question']}")
    if history:
        parts.append("Recent history:")
        parts.append("\n".join(history))
    if parsed:
        # pretty print the parsed_declaration small snippet
        try:
            pretty = json.dumps(parsed, ensure_ascii=False)
        except Exception:
            pretty = str(parsed)
        parts.append("Current parsed_declaration JSON:")
        parts.append(pretty)
    if missing:
        parts.append("Missing fields (need to ask human if required):")
        parts.append(", ".join(missing))

    parts.append("")
    parts.append("Now propose the next single Thought + Action (or final Réponse).")
    # join and return
    return "\n".join(parts)


In [ ]:
def parse_output(output: str) -> Tuple[str, Any, Any]:
    """
    Parse LLM output and return a tuple:
    - ("action", tool_name, tool_args_dict)
    - ("answer", answer_text, None)
    - ("thought", thought_text, None)

    This parser is tolerant:
    - accepts "Action: TOOLNAME" on a line
    - accepts arguments as either JSON after "Arguments:" OR key=value lines
    - if parsing fails for args, returns them as raw string under {"raw": "..."}
    """
    text = output.strip()

    # Try to find an "Action:" line (match up to end-of-line, non-greedy)
    m_action = re.search(r"(?mi)^Action:\s*(?P<tool>[^\n\r]+)", text)
    m_args = re.search(r"(?mi)^Arguments:\s*(?P<args>[\s\S]+)$", text)  # capture until string end

    # If action present, parse args if any
    if m_action:
        tool_name = m_action.group("tool").strip()
        tool_args = {}

        if m_args:
            raw_args = m_args.group("args").strip()
            # Try JSON first
            try:
                parsed = json.loads(raw_args)
                if isinstance(parsed, dict):
                    tool_args = parsed
                else:
                    tool_args = {"raw": parsed}
            except Exception:
                # Fallback: parse key=value lines
                lines = [l.strip() for l in raw_args.splitlines() if l.strip()]
                kv = {}
                for line in lines:
                    # accept "key = value" or "key=value"
                    m_kv = re.match(r"^\s*([^=]+?)\s*=\s*(.+)$", line)
                    if m_kv:
                        key = m_kv.group(1).strip()
                        val = m_kv.group(2).strip()
                        # try to interpret JSON value (numbers, lists, etc.)
                        try:
                            val_parsed = json.loads(val)
                        except Exception:
                            val_parsed = val
                        kv[key] = val_parsed
                    else:
                        # can't parse line -> keep raw under a list
                        kv.setdefault("_raw_lines", []).append(line)
                tool_args = kv if kv else {"raw": raw_args}

        return ("action", tool_name, tool_args)

    # If there's a "Réponse:" or "Answer:" line, treat as final answer
    m_answer = re.search(r"(?mi)^(Réponse|Answer):\s*(?P<ans>[\s\S]+)$", text)
    if m_answer:
        return ("answer", m_answer.group("ans").strip(), None)

    # Try to parse JSON directly as answer/action
    try:
        j = json.loads(text)
        if isinstance(j, dict):
            # if dict contains action key, map to action
            if "action" in j:
                return ("action", j.get("action"), j.get("args", {}))
            if "answer" in j:
                return ("answer", j.get("answer"), None)
    except Exception:
        pass

    # otherwise fallback to thought
    return ("thought", text, None)

In [ ]:
tools = TOOLS
tool_names = list(TOOLS.keys())

def node_thought_action(state: DeclarationReActState) -> DeclarationReActState:
    """
    Node that produces the next Thought/Action/Answer using the LLM.
    It fills last_action/last_arguments when the LLM asks to call a tool,
    or writes the final answer when the LLM produces an 'answer'.
    """
    # Build the prompt using the state's history and some structured context
    # It's helpful to include parsed_declaration and missing fields in the prompt so the LLM
    # can reason clearly about the next step.
    prompt = format_prompt(state, tool_names)
    output = llm_inference(prompt)

    # parse_output must return a tuple like ("action", tool_name, tool_args)
    # or ("answer", answer_text) or ("thought", thought_text)
    step_type, *content = parse_output(output)

    # Append the raw LLM output to history for traceability
    state.setdefault("history", [])
    state["history"].append(f"LLM output: {output}")

    if step_type == "action":
        tool_name, tool_args = content
        # store next action and its arguments
        state["last_action"] = tool_name
        state["last_arguments"] = tool_args or {}
        # keep history friendly: record the action intention
        state["history"].append(f"Action: call tool: {tool_name} with args: {tool_args}")
    elif step_type == "answer":
        # final textual answer produced by the LLM
        state["is_complete"] = True
        state["last_action"] = None
        state["last_arguments"] = None
        state["last_observation"] = None
        state["history"].append(f"Answer: {content[0]}")
    else:
        # Thought only: no action requested, we keep loop running
        state["history"].append(f"Thought: {content[0] if content else ''}")
    return state

def node_tool_execution(state: DeclarationReActState) -> DeclarationReActState:
    """
    Execute the tool stored in state['last_action'] with state['last_arguments'].
    Update state['last_observation'], state['history'], and structured fields:
      - state['parsed_declaration']
      - state['is_complete'], state['missing'] via verify_completeness(parsed_declaration)
    Behavior for ask_human:
      - If parse_declaration tool exists, we call it with a combined raw input that
        contains the old parsed JSON and the new human reply so the LLM can merge them.
      - Otherwise, a simple heuristic fills the first missing field with the reply.
    """
    tool_name = state.get("last_action")
    tool_args = state.get("last_arguments") or {}

    # nothing to execute
    if not tool_name:
        state.setdefault("history", []).append("No action to execute.")
        return state

    # call the tool if available
    if tool_name in TOOLS:
        try:
            observation = TOOLS[tool_name](**tool_args)
        except Exception as e:
            observation = f"Error during tool {tool_name}: {e}"
    else:
        observation = f"Error: Unknown tool {tool_name}"

    # store observation and history
    state["last_observation"] = str(observation)
    state.setdefault("history", []).append(f"Observation from {tool_name}: {state['last_observation']}")

    if tool_name == "parse_declaration":
        if isinstance(observation, dict):
            # Replace entire parsed_declaration with returned dict
            state["parsed_declaration"] = observation

            # After parsing, run verify_completeness if available
            if "verify_completeness" in TOOLS:
                try:
                    verify_res = TOOLS["verify_completeness"](state["parsed_declaration"])
                    if isinstance(verify_res, dict):
                        state["is_complete"] = bool(verify_res.get("is_complete", False))
                        state["missing"] = verify_res.get("missing", [])
                        state["history"].append(f"Auto-verify result: {verify_res}")
                except Exception as e:
                    state["history"].append(f"Auto-verify failed: {e}")
        else:
            state["history"].append("parse_declaration returned non-dict observation.")

    # ---------- CASE 2: verify_completeness tool (explicit call) ----------
    elif tool_name == "verify_completeness":
        if isinstance(observation, dict):
            state["is_complete"] = bool(observation.get("is_complete", False))
            state["missing"] = observation.get("missing", [])
        else:
            state["history"].append("verify_completeness returned unexpected output.")

    # ---------- CASE 3: ask_human tool (human response) ----------
    elif tool_name == "ask_human":
        # observation expected to be the human reply string (or similar)
        human_reply = observation if isinstance(observation, str) else str(observation)
        state["history"].append(f"Human replied: {human_reply}")

        # If parse_declaration tool exists, call it with merged input:
        # Build combined raw input: include previous parsed_declaration JSON and the new human reply.
        if "parse_declaration" in TOOLS and isinstance(state.get("parsed_declaration"), dict):
            # Convert previous parsed_declaration to compact JSON and instruct the LLM to merge
            prev_json = json.dumps(state["parsed_declaration"], ensure_ascii=False)
            combined_raw_input = (
                "Existing parsed JSON:\n" + prev_json + "\n\n"
                "New user input (please update the JSON using this new information):\n"
                + human_reply
                + "\n\n"
                "- If the input contains already a JSON and new information, add the new information to the old JSON and return the new JSON."
            )
            try:
                merged_obs = TOOLS["parse_declaration"](combined_raw_input)
                # If the parse_declaration returns dict, update parsed_declaration and re-run verify
                if isinstance(merged_obs, dict):
                    state["parsed_declaration"] = merged_obs

                    # call verify_completeness automatically
                    if "verify_completeness" in TOOLS:
                        try:
                            verify_res = TOOLS["verify_completeness"](state["parsed_declaration"])
                            if isinstance(verify_res, dict):
                                state["is_complete"] = bool(verify_res.get("is_complete", False))
                                state["missing"] = verify_res.get("missing", [])
                                state["history"].append(f"Auto-verify after human reply: {verify_res}")
                        except Exception as e:
                            state["history"].append(f"Auto-verify failed after human reply: {e}")

                    # Clear asked missing fields or remove those filled by LLM
                    # We keep the current 'missing' returned by verify_completeness.
                else:
                    # If parse_declaration did not return dict, fallback: simple fill
                    if state.get("missing"):
                        first = state["missing"][0]
                        state.setdefault("parsed_declaration", {}).setdefault("extracted", {})[first] = human_reply
                        state["missing"] = state.get("missing", [])[1:]
                        state["history"].append(f"Filled {first} with human reply (fallback).")

            except Exception as e:
                # on error, fallback to naive update
                if state.get("missing"):
                    first = state["missing"][0]
                    state.setdefault("parsed_declaration", {}).setdefault("extracted", {})[first] = human_reply
                    state["missing"] = state.get("missing", [])[1:]
                    state["history"].append(f"Filled {first} with human reply (fallback due to error: {e}).")
        else:
            # No parse tool available -> naive fill into first missing field
            if state.get("missing"):
                first = state["missing"][0]
                state.setdefault("parsed_declaration", {}).setdefault("extracted", {})[first] = human_reply
                state["missing"] = state.get("missing", [])[1:]
                state["history"].append(f"Filled {first} with human reply (no parse tool).")

    # reset action so next Thought node computes next step
    state["last_action"] = None
    state["last_arguments"] = None

    return state

## Graph building

In [ ]:
def build_graph(initial_state: DeclarationReActState):
    graph_builder = StateGraph(initial_state)
    graph_builder.add_node("thought", node_thought_action)
    graph_builder.add_node("action", node_tool_execution)

    graph_builder.add_edge(START, "thought")

    def decide_from_thought(runtime_state: DeclarationReActState):
            if runtime_state.get("is_complete"):
                return END
            if runtime_state.get("last_action"):
                return "action"
            return END

    graph_builder.add_conditional_edges("thought", decide_from_thought)
    graph_builder.add_edge("action", "thought")
    return graph_builder.compile()

In [ ]:

def run_graph(graph, initial_state: Dict[str, Any], max_steps: int = 50):
    """
    Generic runner for the compiled graph.
    - graph: result of build_graph(...). It must provide a `run_once(state)` or we emulate node execution.
    If your StateGraph API differs, adapt accordingly.
    """
    state = initial_state
    step = 0

    # Pretty print function
    def print_new_history(prev_len):
        history = state.get("history", [])
        for line in history[prev_len:]:
            print(line)
        return len(history)

    prev_history_len = 0
    while step < max_steps:
        step += 1
        state = node_thought_action(state)
        prev_history_len = print_new_history(prev_history_len)

        if state.get("is_complete") or state.get("answer"):
            break

        if state.get("last_action"):
            state = node_tool_execution(state)
            prev_history_len = print_new_history(prev_history_len)
            # continue loop, next iteration Thought will run again
        else:
            # if no action and not complete, allow loop to continue (LLM might set action next)
            # small sleep to avoid busy loop in notebook (optional)
            time.sleep(0.01)
            continue

    # final
    print("\n--- FINAL STATE ---")
    print("is_complete:", state.get("is_complete"))
    print("parsed_declaration:", state.get("parsed_declaration"))
    print("missing:", state.get("missing"))
    return state

In [ ]:
initial_state = {
    "question": "Bonjour, on m'a cambriolé ce matin, les voleurs sont passés par le vélux de la "
    "chambre et ont volé tous les appareils électroniques. Merci de me contacter rapidement.",
    "history": [],
    "last_action": None,
    "last_arguments": None,
    "last_observation": None,
    "is_complete": False,
    "parsed_declaration": None,
    "missing": []
}

graph = build_graph(initial_state)

final_state = run_graph(graph, initial_state)